In [1]:
import qpv2

In [2]:
code = r'''
    abort;
    skip;
    [q p] :=0;
    (H \otimes X)[p q];
    assert Pp[p];
    [ pre: Pp[p], post: P1[p]];
    ( skip _ 0.52 \otimes H[p] );
    if (P1[q] \wedge P0[p]) then
        CX[q p]
    else
        while Pm[p] do
            H[p]
        end
    end
'''

In [3]:
res = qpv2.parser.parse(code)
print(res)

abort;
skip;
[q p]:=0;
(H⊗X)[p q];
assert Pp[p];
[ pre: Pp[p], post: P1[p] ];
(
  skip
_0.52⊗
  H[p]
);
if (P1[q] ∧ P0[p]) then
  CX[q p]
else
  while Pm[p] do
    H[p]
  end
end


## Forward Calculation

In [4]:
rho0 = qpv2.OPTParser.parse("c1[]").eval()
print(rho0)

[[1.]][]


In [5]:
code = '''
    [p] :=0;
    [q] :=0;
    skip;
    H[p];
    CX[p q];
    assert P0[p]
'''
prog = qpv2.parser.parse(code)
print(prog)

[p]:=0;
[q]:=0;
skip;
H[p];
CX[p q];
assert P0[p]


In [6]:
print(qpv2.calc(prog, rho0))

[[0.5 0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]][p q]


This code will also terminate.

In [7]:
code = '''
    [p] :=0;
    H[p];
    while P0[p] do
        H[p]
    end
'''
prog = qpv2.parser.parse(code)
print(prog)
print()
print(qpv2.calc(prog, rho0))

[p]:=0;
H[p];
while P0[p] do
  H[p]
end

[[0. 0.]
 [0. 1.]][p]


This code will kill the kernel because it indeed runs forever.

In [8]:
code = '''
    [p] :=0;
    H[p];
    while P0[p] do
        skip
    end
'''
prog = qpv2.parser.parse(code)
print(prog)
print()

# this execution kills the kernel
# print(qpv2.calc(prog, rho0))

[p]:=0;
H[p];
while P0[p] do
  skip
end



## Refinement Program Statement

In [9]:
code = '''
    [pre: P0[p], post: P0[p]]
         = RSKIP => 
      skip
'''

prog = qpv2.parser.parse(code)
print(prog)

rho0 = qpv2.OPTParser.parse("c1[]").eval()
print(qpv2.calc(prog, rho0))

[ pre: P0[p], post: P0[p] ]
  = RSKIP[] => {
skip }
[[1.]][]


In [10]:
code = '''
    [pre: P0[p], post: I[p]]
         = RIMPLY => 
    [pre: I[p], post : I[p]]
         = RSKIP => 
      skip
'''

prog = qpv2.parser.parse(code)
print(prog)
print()
rho0 = qpv2.OPTParser.parse("c1[]").eval()
print(qpv2.calc(prog, rho0))

[ pre: P0[p], post: I[p] ]
  = RIMPLY[] => {
[ pre: I[p], post: I[p] ] }
  = RSKIP[] => {
skip }

[[1.]][]


In [11]:

code = '''
    [pre: P0[p], post: P0[p]]
        = RSEQ =>
    [pre: P0[p], post: P0[p]]; 
    [pre: P0[p], post: P0[p]]
'''

prog = qpv2.parser.parse(code)
print(prog)

[ pre: P0[p], post: P0[p] ]
  = RSEQ[] => {
[ pre: P0[p], post: P0[p] ];
[ pre: P0[p], post: P0[p] ] }


Note: A problem of extract exists here: we also need to extract the head of proof chain.

In [12]:
code = '''
    [pre: P0[p], post: P0[p]]
        = RSEQ =>
    {
        [pre: P0[p], post: P0[p]]
            = RSKIP =>
        skip
    }; 
    [pre: P0[p], post: P0[p]]
'''

prog = qpv2.parser.parse(code)
print(prog)
print()
print(prog.extract)

[ pre: P0[p], post: P0[p] ]
  = RSEQ[] => {
[ pre: P0[p], post: P0[p] ]
  = RSKIP[] => {
skip };
[ pre: P0[p], post: P0[p] ] }

skip;
[ pre: P0[p], post: P0[p] ]
